In [2]:
# Sudoku example.

# The Sudoku board is a 9x9 grid, which is further divided into a 3x3 grid
# of 3x3 grids.  Each cell in the grid must take a value from 0 to 9.
# No two grid cells in the same row, column, or 3x3 subgrid may take the
# same value.
#
# In the MIP formulation, binary variables x[i,j,v] indicate whether
# cell <i,j> takes value 'v'.  The constraints are as follows:
#   1. Each cell must take exactly one value (sum_v x[i,j,v] = 1)
#   2. Each value is used exactly once per row (sum_i x[i,j,v] = 1)
#   3. Each value is used exactly once per column (sum_j x[i,j,v] = 1)
#   4. Each value is used exactly once per 3x3 subgrid (sum_grid x[i,j,v] = 1)
#
# Input datasets for this example can be found in examples/data/sudoku*.

import sys
import math
from gurobipy import *


f = open('sudoku_input.txt')

grid = f.read().split()

f.close()

n = len(grid[0])
s = int(math.sqrt(n))


# Create our 3-D array of model variables

model = Model('sudoku')

vars = model.addVars(n,n,n, vtype=GRB.BINARY, name='G')


# Fix variables associated with cells whose values are pre-specified

for i in range(n):
    for j in range(n):
        if grid[i][j] != '.':
            v = int(grid[i][j]) - 1
            vars[i,j,v].LB = 1

# Each cell must take one value

model.addConstrs((vars.sum(i,j,'*') == 1
                 for i in range(n)
                 for j in range(n)), name='V')

# Each value appears once per row

model.addConstrs((vars.sum(i,'*',v) == 1
                 for i in range(n)
                 for v in range(n)), name='R')

# Each value appears once per column

model.addConstrs((vars.sum('*',j,v) == 1
                 for j in range(n)
                 for v in range(n)), name='C')


# Each value appears once per subgrid

model.addConstrs((
    quicksum(vars[i,j,v] for i in range(i0*s, (i0+1)*s)
                         for j in range(j0*s, (j0+1)*s)) == 1
             for v in range(n)
             for i0 in range(s)
             for j0 in range(s)), name='Sub')

model.optimize()

model.write('sudoku.lp')

print('')
print('Solution:')
print('')

# Retrieve optimization result

solution = model.getAttr('X', vars)

for i in range(n):
    sol = ''
    for j in range(n):
        for v in range(n):
            if solution[i,j,v] > 0.5:
                sol += str(v+1)
    print(sol)

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 324 rows, 729 columns and 2916 nonzeros
Model fingerprint: 0x03827106
Variable types: 0 continuous, 729 integer (729 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 324 rows and 729 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%

Solution:

928476351
514839627
763512984
281793546
375164892
649258173
457381269
836927415
192645738
